In [12]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(project_id='54fcXXX5-XXXX-XXXX-XXXX-0175d16a8077', project_access_token='p-37d9d3f68efc8fXXXXXXXX938bf0XXXXXX5e064b7')
pc = project.project_context


In [13]:
PROJECT_ID='54fcXXX5-XXXX-XXXX-XXXX-0175d16a8077'
PROJECT_ACCESS_TOKEN='p-37d9d3f68efc8fXXXXXXXX938bf0XXXXXX5e064b7'
COS_ENDPOINT = project.get_metadata()['entity']['storage']['properties']['endpoint_url']
COS_ACCESS_KEY_ID = project.get_metadata()['entity']['storage']['properties']['credentials']['editor']['access_key_id']
COS_SECRET_ACCESS_KEY = project.get_metadata()['entity']['storage']['properties']['credentials']['editor']['secret_access_key']
BUCKET_NAME = project.get_project_bucket_name()

In [14]:
import boto3, sys, json
from project_lib.utils import ProjectHandle
from project_lib.proxies import APIProxyFactory
local_handle = ProjectHandle(PROJECT_ID,PROJECT_ACCESS_TOKEN)
local_api_proxy_factory = APIProxyFactory(local_handle)
local_assetsAPI = local_api_proxy_factory.newAssetsAPI()
notebook_list = []
for nb in project.get_assets():
    if (nb.get("type") == 'notebook'):
        local_nb_asset = local_assetsAPI.fetch_asset(nb['asset_id'])
        notebook_list.append([local_nb_asset['metadata'].get("name"), local_nb_asset['attachments'][0].get("object_key")])
cos_client = boto3.client('s3', aws_access_key_id=COS_ACCESS_KEY_ID, aws_secret_access_key=COS_SECRET_ACCESS_KEY,endpoint_url=COS_ENDPOINT)
for nb in notebook_list:
   file = cos_client.get_object(Bucket = BUCKET_NAME, Key = nb[1])
   jfile = json.loads(file["Body"].read())
   script_filename = nb[0] + '.py'
   script_file = open(script_filename, 'w') 
   script_file.write("#COS-filename "+nb[1]+"\n")
   if jfile["nbformat"] >=4:
      for i,cell in enumerate(jfile["cells"]):
         script_file.write("#cell "+str(i)+"\n")
         for line in cell["source"]:
            script_file.write(line)
         script_file.write('\n\n')
   else:
      for i,cell in enumerate(j["worksheets"][0]["cells"]):
         script_file.write("#cell "+str(i)+"\n")
         for line in cell["input"]:
             script_file.write(line)
         script_file.write('\n\n')
   script_file.close() 

In [15]:
!ls -lrt

total 12
-rw-rw---- 1 wsuser watsonstudio 2483 Aug 31 19:43 Notebook-Main.py
-rw-rw---- 1 wsuser watsonstudio  200 Aug 31 19:43 Script-Notebook1.py
-rw-rw---- 1 wsuser watsonstudio  168 Aug 31 19:43 SubNotebook2-Python.py


In [16]:
%run Script-Notebook1.py

Script Notebook 1
Script 1 completed!


In [17]:
simple_function()

54987


2

In [18]:
%run SubNotebook2-Python.py

Script SubNotebook2


In [19]:
!cat Script-Notebook1.py

#COS-filename notebook/Script-Notebook1_MudxEbxxc.ipynb
#cell 0
print ("Script Notebook 1")
countItens = 3
def simple_function():
    print(54987)
    return 2

#cell 1
print ("Script 1 completed!")

